# 03 - Recommender System

In [63]:
# imports
import pandas as pd
import numpy as np
import time

from scipy import sparse # cut down on memory size
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine

pd.options.display.max_columns = 35

___

In [2]:
# read in the data
df = pd.read_csv('../Data/cards_cleaned.csv')
df.head()

,name,layout,colors,color_identity,mana_cost,cmc,type_line,card_type,super_type,sub_type,oracle_text,oracle_text_token,legalities,rarity,power,toughness,loyalty,card_faces,activated_ability,triggered_ability,oracle_text_back,oracle_text_back_token,colors_back,power_back,toughness_back,loyalty_back,card_type_back,super_type_back,sub_type_back,mana_cost_back
0,Static Orb,normal,[],[],{3},3.0,Artifact,Artifact,NONE,NONE,"As long as Static Orb is untapped, players can...",as long as static orb is untapped players can'...,legacy vintage commander duel,rare,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
1,Sensory Deprivation,normal,['U'],['U'],{U},1.0,Enchantment — Aura,Enchantment,NONE,Aura,Enchant creature Enchanted creature gets -3/-0.,enchant creature enchanted creature gets -3/-0,pioneer modern legacy pauper vintage penny com...,common,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
2,Road of Return,normal,['G'],['G'],{G}{G},2.0,Sorcery,Sorcery,NONE,NONE,Choose one — • Return target permanent card fr...,choose one return target permanent card from y...,legacy vintage commander duel,rare,NONE,NONE,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
3,Storm Crow,normal,['U'],['U'],{1}{U},2.0,Creature — Bird,Creature,NONE,Bird,Flying (This creature can't be blocked except ...,flying this creature can't be blocked except b...,modern legacy pauper vintage penny commander duel,common,1,2,NONE,NONE,0.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
4,Walking Sponge,normal,['U'],['U'],{1}{U},2.0,Creature — Sponge,Creature,NONE,Sponge,{T}: Target creature loses your choice of flyi...,{t} target creature loses your choice of flyin...,legacy vintage commander duel,uncommon,1,1,NONE,NONE,1.0,0.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


In [3]:
df.shape

(18098, 30)

___

## Oracle text recommender system
To start I want to build a recommender system that just looks at oracle text. In order to do that I need to combine both oracle_text and oracle_text_back into one single column

In [4]:
df['oracle_combined'] = df['oracle_text_token'] + " " + df['oracle_text_back_token']

In [5]:
# start by isolating the name of the card and it's oracle_text
oracle = df['oracle_combined']

# vectorize all our words
cvec = CountVectorizer(stop_words=['none'],
                      min_df=2,
                      max_df=.99,
                      ngram_range=(1,6),
                      token_pattern="[a-zA-Z{}+'0-9-/−]+") # we should use the same RegEx to keep certain characters together 

oracle_vec = cvec.fit_transform(oracle)

# convert to a dataframe so we can use this later on as well
converted_df = pd.DataFrame(oracle_vec.toarray(), columns=cvec.get_feature_names(), index=df['name'])

In [6]:
converted_df.shape

(18098, 130022)

This is a whole lot of features to examine so I'm going to use Variance Threshold to drop some n_gram columns that only exist in a small number of cards

In [7]:
selector = VarianceThreshold(.001)
new_array = selector.fit_transform(converted_df)

new_array.shape

(18098, 15254)

In [8]:
# get the list of columns kept
vt_list = converted_df.columns[selector.get_support()]

# update our converted_df with just the important columns
converted_df = converted_df[vt_list]

For the recommender system to work efficiently we need to convert the data back into a sparse matrix with the new names as the index

In [9]:
sparse_df = sparse.csr_matrix(converted_df)

In [10]:
# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')

In [11]:
rec.shape

(18098, 18098)

In [12]:
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,Marang River Prowler,Aura Graft,...,Trinket Mage,Skyshroud Blessing,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,1.000000,1.000000,0.976967,1.000000,0.916955,0.976224,1.000000,1.000000,1.000000,0.897467,0.979617,0.982608,0.962356,1.000000,0.771425,1.000000,...,1.000000,0.977805,0.987648,0.800691,0.983457,0.975825,0.980091,1.000000,1.000000,0.922746,1.000000,1.000000,1.0,0.985455,1.000000,0.944326,1.000000
Sensory Deprivation,1.000000,0.000000,1.000000,0.939834,0.921311,1.000000,1.000000,0.952886,1.000000,1.000000,0.625030,0.946757,0.954569,1.000000,1.000000,1.000000,0.962576,...,1.000000,1.000000,1.000000,0.869842,0.956786,1.000000,0.895990,1.000000,0.943989,1.000000,1.000000,1.000000,1.0,0.962006,0.947074,0.927285,1.000000
Road of Return,1.000000,1.000000,0.000000,1.000000,0.925013,1.000000,0.917141,0.910205,1.000000,0.952435,0.979581,0.979705,0.792190,0.900045,0.869256,0.852062,0.928673,...,0.790645,0.966850,0.858557,0.909041,0.942347,0.855568,0.970265,0.800785,0.925274,0.974359,0.911157,0.692745,1.0,0.927588,0.949564,0.854482,1.000000
Storm Crow,0.976967,0.939834,1.000000,0.000000,0.919516,1.000000,1.000000,0.927716,1.000000,1.000000,0.958908,0.959156,0.976766,0.907804,1.000000,0.877859,1.000000,...,0.980629,0.940700,0.991749,0.789214,0.966850,0.983852,0.946808,1.000000,1.000000,0.982799,0.981662,1.000000,1.0,0.970855,0.959400,0.972110,0.962602
Walking Sponge,1.000000,0.921311,0.925013,0.919516,0.000000,1.000000,0.937689,0.952731,0.906341,1.000000,0.946257,0.928776,0.848066,0.791724,0.971323,0.960064,0.924906,...,0.949331,0.728552,0.913674,0.941962,0.956645,0.915522,0.930432,0.961160,0.887610,1.000000,0.916058,0.960064,1.0,0.809408,0.911501,0.768979,1.000000


In [13]:
# now to test the recommender system
rec_df['Shock'].sort_values()[0:11]

name
Shock                0.000000
Unyaro Bee Sting     0.000000
Tarfire              0.000000
Magma Jet            0.058487
Deadapult            0.133975
Shock Troops         0.133975
Moonglove Extract    0.133975
Seismic Assault      0.149037
Blood Rites          0.149037
Crackling Triton     0.149037
Goblin Test Pilot    0.149037
Name: Shock, dtype: float64

In [14]:
rec_df['Static Orb'].sort_values()[0:11]

name
Static Orb        0.000000
Winter Orb        0.099500
Damping Field     0.360979
Imi Statue        0.360979
Stoic Angel       0.442914
Castle Raptors    0.543250
Storage Matrix    0.544510
Kill Switch       0.578152
Giant Tortoise    0.594660
Watchdog          0.627065
Eyekite           0.628609
Name: Static Orb, dtype: float64

In [15]:
rec_df['Prized Amalgam'].sort_values()[0:11]

name
Prized Amalgam            0.000000
Bone Dragon               0.367289
Reassembling Skeleton     0.374905
Footsteps of the Goryo    0.378368
Chronosavant              0.408758
Scrapheap Scrounger       0.409055
Apprentice Necromancer    0.414345
Wake the Dead             0.418516
Despoiler of Souls        0.418516
Ghoulsteed                0.424205
Cauldron Dance            0.424953
Name: Prized Amalgam, dtype: float64

In [16]:
rec_df['Wrath of God'].sort_values()[0:11]

name
Wrath of God               0.000000
Damnation                  0.000000
Perish                     0.117647
Winds of Rath              0.153190
Shatterstorm               0.185908
Plague Wind                0.207882
Abu Ja'far                 0.223886
Retribution of the Meek    0.344064
Do or Die                  0.348305
Obliterate                 0.356079
Child of Alara             0.356404
Name: Wrath of God, dtype: float64

In [17]:
rec_df['Grizzly Bears'].sort_values()[0:11] # vanilla creature

name
Grizzly Bears                   0.0
Static Orb                      1.0
Aerial Volley                   1.0
Explosive Apparatus             1.0
Spontaneous Mutation            1.0
Rakdos Locket                   1.0
Voltaic Brawler                 1.0
Jousting Lance                  1.0
Tatsumasa, the Dragon's Fang    1.0
Deadeye Tormentor               1.0
Fiery Hellhound                 1.0
Name: Grizzly Bears, dtype: float64

This is a great start! Now I want to add the numerical features and see the results
___

## Adding numerical data to our features

In [18]:
df.dtypes

name                       object
layout                     object
colors                     object
color_identity             object
mana_cost                  object
cmc                       float64
type_line                  object
card_type                  object
super_type                 object
sub_type                   object
oracle_text                object
oracle_text_token          object
legalities                 object
rarity                     object
power                      object
toughness                  object
loyalty                    object
card_faces                 object
activated_ability         float64
triggered_ability         float64
oracle_text_back           object
oracle_text_back_token     object
colors_back                object
power_back                 object
toughness_back             object
loyalty_back               object
card_type_back             object
super_type_back            object
sub_type_back              object
mana_cost_back

In [19]:
# we already have a df for the oracle text so we can just use that one and add to it.
converted_df['cmc'] = df['cmc'].values
converted_df['activated_ability'] = df['activated_ability'].values
converted_df['triggered_ability'] = df['triggered_ability'].values

# convert to a sparse matrix
sparse_df = sparse.csr_matrix(converted_df)

# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')

# turn into a dataframe for interpretability
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,Marang River Prowler,Aura Graft,...,Trinket Mage,Skyshroud Blessing,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,0.885292,0.911516,0.863296,0.851569,0.539805,0.720115,0.953171,0.595071,0.844143,0.763221,0.831784,0.854321,0.934378,0.774718,0.644097,0.856491,...,0.889318,0.867995,0.864715,0.706802,0.824260,0.758095,0.880962,0.887022,0.757875,0.804900,0.831684,0.746309,0.513336,0.912390,0.734547,0.814129,0.748688
Sensory Deprivation,0.885292,0.000000,0.957145,0.886499,0.856223,0.794262,0.872918,0.931959,0.803884,0.924515,0.582248,0.877794,0.894167,0.984109,0.890891,0.918350,0.895743,...,0.946394,0.945200,0.939519,0.817426,0.882149,0.891852,0.851750,0.945282,0.835827,0.937006,0.918481,0.877132,0.764298,0.927261,0.828580,0.864968,0.878284
Road of Return,0.911516,0.957145,0.000000,0.956223,0.875228,0.841296,0.833349,0.895027,0.848718,0.898100,0.924177,0.924585,0.755084,0.889676,0.803614,0.800552,0.879366,...,0.758787,0.926024,0.819214,0.866988,0.886364,0.791440,0.933291,0.767853,0.846222,0.927111,0.855369,0.649317,0.818182,0.901807,0.859504,0.811206,0.906109
Storm Crow,0.863296,0.886499,0.956223,0.000000,0.853131,0.789837,0.870185,0.907327,0.799666,0.922891,0.887040,0.887649,0.913512,0.894487,0.888544,0.805387,0.929000,...,0.926989,0.888043,0.930495,0.740971,0.889646,0.875717,0.899041,0.944106,0.880212,0.919564,0.900073,0.874489,0.759228,0.934984,0.835837,0.905169,0.844583
Walking Sponge,0.851569,0.856223,0.875228,0.853131,0.000000,0.733777,0.786224,0.911955,0.661636,0.902323,0.856908,0.826053,0.767189,0.784090,0.835283,0.859128,0.842607,...,0.884391,0.680903,0.843475,0.868750,0.847501,0.790087,0.872110,0.893794,0.757213,0.918486,0.820675,0.809211,0.695003,0.764690,0.764320,0.694222,0.842500


In [20]:
# now to test the recommender system
rec_df['Shock'].sort_values()[0:11]

name
Shock                0.000000e+00
Tarfire              4.440892e-16
Magma Jet            6.580127e-02
Unyaro Bee Sting     1.237505e-01
Arc Trail            1.691324e-01
Moonglove Extract    1.699426e-01
Deadapult            1.699426e-01
Seal of Fire         1.708438e-01
Ember Hauler         1.711375e-01
Orcish Vandal        1.711375e-01
Seismic Assault      1.806535e-01
Name: Shock, dtype: float64

In [21]:
rec_df['Static Orb'].sort_values()[0:11]

name
Static Orb           0.000000
Winter Orb           0.095126
Imi Statue           0.264233
Damping Field        0.264233
Stoic Angel          0.315962
Castle Raptors       0.371254
Wardscale Dragon     0.423140
Storage Matrix       0.453183
Mungha Wurm          0.454499
Scoria Cat           0.454827
Rampaging Cyclops    0.455752
Name: Static Orb, dtype: float64

In [22]:
rec_df['Prized Amalgam'].sort_values()[0:11]

name
Prized Amalgam            0.000000
Footsteps of the Goryo    0.352702
Reassembling Skeleton     0.362452
Bone Dragon               0.368833
Scrapheap Scrounger       0.392484
Cauldron Dance            0.397659
Apprentice Necromancer    0.397741
Wake the Dead             0.398827
Despoiler of Souls        0.401777
Stitchwing Skaab          0.424360
Skyfire Phoenix           0.425799
Name: Prized Amalgam, dtype: float64

In [23]:
rec_df['Wrath of God'].sort_values()[0:11]

name
Wrath of God       0.000000
Damnation          0.000000
Winds of Rath      0.076011
Perish             0.078235
Shatterstorm       0.093307
Plague Wind        0.116612
Obliterate         0.167576
Catastrophe        0.182008
Day of Judgment    0.183503
Child of Alara     0.187596
Mageta the Lion    0.225403
Name: Wrath of God, dtype: float64

In [24]:
rec_df['Jace, the Mind Sculptor'].sort_values()[0:11]

name
Jace, the Mind Sculptor    0.000000
Coral Fighters             0.374088
Voyage's End               0.389159
Brainstorm                 0.413580
Riverwise Augur            0.415442
Select for Inspection      0.432234
Eye Spy                    0.434878
Anchor to the Aether       0.438873
Precognition               0.443417
Dream Cache                0.450290
Cavalier of Gales          0.453835
Name: Jace, the Mind Sculptor, dtype: float64

In [25]:
rec_df['Grizzly Bears'].sort_values()[0:11] # vanilla creature

name
Raging Bull                  0.0
Alpine Grizzly               0.0
Shatterskull Giant           0.0
Mass of Ghouls               0.0
Terror of the Fairgrounds    0.0
Panther Warriors             0.0
Broodhunter Wurm             0.0
Obsidian Giant               0.0
Flameborn Viron              0.0
Sworn Guardian               0.0
Canyon Minotaur              0.0
Name: Grizzly Bears, dtype: float64

___

## Non-numerical data
Now I have to convert the non-numerical data into numerical data

### Power / Toughness / Loyalty

Through some outside reasearch on ScryFall and how other sources interpret * s in power and toughness, I'm going to impute the * s as 0 and * +1s as 1 for both power and toughness. Same is true for loyalty

In [26]:
# power
zero_power_index = df.loc[(df['power'] == '*')].index
zero_power_back_index = df.loc[(df['power_back'] == '*')].index
df.loc[zero_power_index, 'power'] = 0
df.loc[zero_power_back_index, 'power_back'] = 0

one_power_index = df.loc[(df['power'] == '1+*')].index
df.loc[one_power_index, 'power'] = 1

# toughness
zero_toughness_index = df.loc[(df['toughness'] == '*')].index
zero_toughness_back_index = df.loc[(df['toughness_back'] == '*')].index
df.loc[zero_toughness_index, 'toughness'] = 0
df.loc[zero_toughness_back_index, 'toughness_back'] = 0

one_toughness_index = df.loc[(df['toughness'] == '1+*')].index
df.loc[one_toughness_index, 'toughness'] = 1

# loyalty
zero_loyalty_index = df.loc[(df['loyalty'] == 'X')].index
zero_loyalty_back_index = df.loc[(df['loyalty_back'] == 'X')].index
df.loc[zero_loyalty_index, 'loyalty'] = 0
df.loc[zero_loyalty_back_index, 'loyalty_back'] = 0

In [27]:
df.dtypes

name                       object
layout                     object
colors                     object
color_identity             object
mana_cost                  object
cmc                       float64
type_line                  object
card_type                  object
super_type                 object
sub_type                   object
oracle_text                object
oracle_text_token          object
legalities                 object
rarity                     object
power                      object
toughness                  object
loyalty                    object
card_faces                 object
activated_ability         float64
triggered_ability         float64
oracle_text_back           object
oracle_text_back_token     object
colors_back                object
power_back                 object
toughness_back             object
loyalty_back               object
card_type_back             object
super_type_back            object
sub_type_back              object
mana_cost_back

In [28]:
# add the newly made numerical columns to the converted df
converted_df['power'] = df['power'].values
converted_df['power_back'] = df['power_back'].values
converted_df['toughness'] = df['toughness'].values
converted_df['toughness_back'] = df['toughness_back'].values
converted_df['loyalty'] = df['loyalty'].values
converted_df['loyalty_back'] = df['loyalty_back'].values

In [29]:
converted_df.head()

,+0/+1,+0/+1 until,+0/+1 until end,+0/+1 until end of,+0/+1 until end of turn,+0/+2,+0/+3,+1,+1/+0,+1/+0 and,+1/+0 and gains,+1/+0 and has,+1/+0 for,+1/+0 for each,+1/+0 until,+1/+0 until end,+1/+0 until end of,...,{w} {t},{w}{u}{b}{r}{g},{w}{w},{x},{x} {t},−1,−2,−3,−6,−7,−8,cmc,activated_ability,triggered_ability,power_back,toughness_back,loyalty_back
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,3.0,0.0,0.0,NONE,NONE,NONE
Sensory Deprivation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,NONE,NONE,NONE
Road of Return,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,NONE,NONE,NONE
Storm Crow,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2.0,0.0,0.0,NONE,NONE,NONE
Walking Sponge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2.0,1.0,0.0,NONE,NONE,NONE


___

In [30]:
# set the index of df to be the same as the converted_df; the name of the card
df = df.set_index(df['name']).drop(columns='name')

In [31]:
# essentially makeing dummy variables for non-numerical data

# initialize some lists to check
wburg = ['B', 'G', 'R', 'U', 'W']

card_types = ['Creature', 'Instant', 'Enchantment', 'Sorcery', 'Artifact', 'Land', 'Planeswalker', 'Tribal']

subtypes = list(set(" ".join(df['sub_type'].value_counts().index).split()))
subtypes.remove('NONE')

super_types = list(set(" ".join(df['super_type'].value_counts().index).split()))
super_types.remove('NONE')

formats = list(set(" ".join(df['legalities'].value_counts().index).split()))
formats.remove('NONE')

rarities = ['common', 'uncommon', 'rare', 'mythic']


# timer and progress checker
t0 = time.time()
counter = 0

for counter, index in enumerate(converted_df.index):
    # colors
    for color in wburg:
        converted_df.loc[index, 'colors_' + color] = (color in df.loc[index, 'colors']) * 1
        converted_df.loc[index, 'color_identity_' + color] = (color in df.loc[index, 'color_identity']) * 1
    
    # card_types
    for ctype in card_types:
        converted_df.loc[index, 'card_type_' + ctype] = (ctype in df.loc[index, 'card_type']) * 1
        
    # sub_types
    for stype in subtypes:
        converted_df.loc[index, 'sub_type_' + stype] = (stype in df.loc[index, 'sub_type']) * 1
        
    # super_type
    for st in super_types:
        converted_df.loc[index, 'super_type_' + st] = (st in df.loc[index, 'super_type']) * 1
    
    # legalities
    for form in formats:
        converted_df.loc[index, 'legalities_' + form] = (form in df.loc[index, 'legalities']) * 1
    
    # rarity
    for r in rarities:
        converted_df.loc[index, 'rarity_' + r] = (r in df.loc[index, 'rarity']) * 1
    
    # progress checker
    if counter % 1000 == 0:
        print(f'Converted {counter} cards out of a total of {len(converted_df.index)}')
        print(f'mins: {(time.time() - t0)/60}')
        print('-------------------------------')

Converted 0 cards out of a total of 18098
mins: 0.304518469174703
-------------------------------
Converted 1000 cards out of a total of 18098
mins: 4.218396484851837
-------------------------------
Converted 2000 cards out of a total of 18098
mins: 8.541768070062002
-------------------------------
Converted 3000 cards out of a total of 18098
mins: 12.34899011850357
-------------------------------
Converted 4000 cards out of a total of 18098
mins: 16.255391617616016
-------------------------------
Converted 5000 cards out of a total of 18098
mins: 20.066371218363443
-------------------------------
Converted 6000 cards out of a total of 18098
mins: 23.877162885665893
-------------------------------
Converted 7000 cards out of a total of 18098
mins: 27.6874542872111
-------------------------------
Converted 8000 cards out of a total of 18098
mins: 31.22906694809596
-------------------------------
Converted 9000 cards out of a total of 18098
mins: 34.73920306762059
-----------------------

In [32]:
# convert 'NONE's to np.NaN's so we can scale our data then impute all the NaNs
converted_df = converted_df.replace('NONE', np.NaN)

# convert those columns to numerical data
converted_df['power_back'] = converted_df['power_back'].astype(float)
converted_df['power'] = converted_df['power'].astype(float)
converted_df['toughness'] = converted_df['toughness'].astype(float)
converted_df['toughness_back'] = converted_df['toughness_back'].astype(float)
converted_df['loyalty'] = converted_df['loyalty'].astype(float)
converted_df['loyalty_back'] = converted_df['loyalty_back'].astype(float)

Now that we have everything coverted to numerical data, we should scale the non-binarized features so everything will be weighted equally. This means we should just scale the columns that have a range of values, e.g. power, toughness, loyalty, and cmc.

In [33]:
range_col_list = ['power', 'power_back', 'toughness', 'toughness_back', 'loyalty', 'loyalty_back', 'cmc']
converted_df[range_col_list]

,power,power_back,toughness,toughness_back,loyalty,loyalty_back,cmc
name,,,,,,,
Static Orb,NaN,NaN,NaN,NaN,NaN,NaN,3.0
Sensory Deprivation,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Road of Return,NaN,NaN,NaN,NaN,NaN,NaN,2.0
Storm Crow,1.0,NaN,2.0,NaN,NaN,NaN,2.0
Walking Sponge,1.0,NaN,1.0,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...
Devoted Hero,1.0,NaN,2.0,NaN,NaN,NaN,1.0
Without Weakness,NaN,NaN,NaN,NaN,NaN,NaN,2.0
Firesong and Sunspeaker,4.0,NaN,6.0,NaN,NaN,NaN,6.0


In [34]:
ss = StandardScaler()
range_col_sc = ss.fit_transform(converted_df[range_col_list])

In [35]:
sc_df = pd.DataFrame(range_col_sc, index = df.index.values, columns=range_col_list)

In [36]:
sc_df.head()

,power,power_back,toughness,toughness_back,loyalty,loyalty_back,cmc
Static Orb,NaN,NaN,NaN,NaN,NaN,NaN,-0.171423
Sensory Deprivation,NaN,NaN,NaN,NaN,NaN,NaN,-1.300191
Road of Return,NaN,NaN,NaN,NaN,NaN,NaN,-0.735807
Storm Crow,-0.901216,NaN,-0.41462,NaN,NaN,NaN,-0.735807
Walking Sponge,-0.901216,NaN,-0.99908,NaN,NaN,NaN,-0.735807


In [37]:
sc_df = sc_df.fillna(0)

In [38]:
sc_df.head()

,power,power_back,toughness,toughness_back,loyalty,loyalty_back,cmc
Static Orb,0.000000,0.0,0.00000,0.0,0.0,0.0,-0.171423
Sensory Deprivation,0.000000,0.0,0.00000,0.0,0.0,0.0,-1.300191
Road of Return,0.000000,0.0,0.00000,0.0,0.0,0.0,-0.735807
Storm Crow,-0.901216,0.0,-0.41462,0.0,0.0,0.0,-0.735807
Walking Sponge,-0.901216,0.0,-0.99908,0.0,0.0,0.0,-0.735807


In [39]:
converted_df = pd.concat([converted_df.fillna(0).drop(columns=range_col_list), sc_df], axis=1)

In [40]:
# convert to a sparse matrix
sparse_df = sparse.csr_matrix(converted_df)

In [41]:
converted_df.isnull().sum().sum()

0

In [42]:
t0 = time.time()
# build the recommender system using cosine similarity
rec = pairwise_distances(sparse_df, metric='cosine')
print((time.time() - t0)/60)

0.6737643798192342


In [43]:
# turn into a dataframe for interpretability
rec_df = pd.DataFrame(rec, index=converted_df.index, columns=converted_df.index)
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,Marang River Prowler,Aura Graft,...,Trinket Mage,Skyshroud Blessing,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,0.873256,0.922700,0.902261,0.902714,0.776722,0.902974,0.936812,0.834946,0.879136,0.845122,0.914283,0.909250,0.920169,0.900858,0.736824,0.903822,...,0.948871,0.903492,0.946999,0.755334,0.916317,0.878582,0.914390,0.947983,0.931651,0.823542,0.929009,0.929058,0.807054,0.934629,0.922868,0.903576,0.858726
Sensory Deprivation,0.873256,0.000000,0.921416,0.740269,0.753184,0.743237,0.899692,0.800355,0.732003,0.777973,0.509072,0.834738,0.901527,0.910635,0.761418,0.798380,0.755995,...,0.876959,0.862302,0.915397,0.785160,0.874106,0.903407,0.772500,0.888395,0.817251,0.886838,0.939183,0.868650,0.630570,0.839691,0.896963,0.878733,0.713580
Road of Return,0.922700,0.921416,0.000000,0.954260,0.881166,0.886467,0.880287,0.881781,0.867445,0.868711,0.944494,0.944848,0.752530,0.865972,0.842397,0.830297,0.882519,...,0.777963,0.905112,0.837042,0.885204,0.897840,0.819342,0.915827,0.774108,0.903097,0.902026,0.888863,0.684497,0.880370,0.902046,0.923654,0.828494,0.917864
Storm Crow,0.902261,0.740269,0.954260,0.000000,0.783188,0.856448,0.929037,0.828272,0.778113,0.830431,0.849624,0.885632,0.918470,0.867444,0.846572,0.764191,0.867024,...,0.893395,0.867449,0.939793,0.735358,0.918832,0.935606,0.846428,0.917411,0.925078,0.919631,0.940725,0.926406,0.727157,0.901698,0.931513,0.937399,0.753760
Walking Sponge,0.902714,0.753184,0.881166,0.783188,0.000000,0.884652,0.879105,0.805143,0.649754,0.817888,0.856545,0.827009,0.784286,0.763053,0.852932,0.807806,0.777807,...,0.865633,0.680594,0.873182,0.859729,0.916414,0.869660,0.815832,0.895287,0.833769,0.918551,0.896560,0.905671,0.691195,0.773115,0.916976,0.761958,0.763824


In [44]:
# now to test the recommender system
rec_df['Shock'].sort_values()[1:11]

name
Magma Jet              0.094759
Tarfire                0.107693
Ember Hauler           0.172588
Seal of Fire           0.204892
Explosive Apparatus    0.222489
Unyaro Bee Sting       0.237035
Moonglove Extract      0.238164
Arc Trail              0.245916
Molten Vortex          0.246273
Shock Troops           0.247442
Name: Shock, dtype: float64

In [45]:
rec_df['Lightning Bolt'].sort_values()[1:11]

name
Searing Spear            0.075160
Lightning Strike         0.085059
Volcanic Hammer          0.104100
Open Fire                0.116373
Ghostfire                0.131822
Precision Bolt           0.137363
Valakut Invoker          0.170199
Mudbutton Torchrunner    0.263050
Lightning Helix          0.291133
Pitchburn Devils         0.292504
Name: Lightning Bolt, dtype: float64

In [46]:
rec_df['Static Orb'].sort_values()[1:11]

name
Winter Orb        0.086520
Imi Statue        0.316573
Damping Field     0.403254
Stoic Angel       0.472754
Storage Matrix    0.487274
Kill Switch       0.509889
Stabilizer        0.564591
Castle Raptors    0.570254
Watchdog          0.577100
Giant Tortoise    0.597442
Name: Static Orb, dtype: float64

In [47]:
rec_df['Prized Amalgam'].sort_values()[1:11]

name
Footsteps of the Goryo    0.379864
Bone Dragon               0.389764
Wake the Dead             0.409315
Scrapheap Scrounger       0.410393
Despoiler of Souls        0.412587
Apprentice Necromancer    0.415171
Cauldron Dance            0.433473
Reassembling Skeleton     0.438801
Brackwater Elemental      0.444423
Ghoulsteed                0.445311
Name: Prized Amalgam, dtype: float64

In [48]:
rec_df['Wrath of God'].sort_values()[1:11]

name
Damnation                  0.076469
Winds of Rath              0.142768
Perish                     0.236058
Day of Judgment            0.238803
Shatterstorm               0.253226
Retribution of the Meek    0.278159
Plague Wind                0.287472
Catastrophe                0.299201
Abu Ja'far                 0.350953
Obliterate                 0.351582
Name: Wrath of God, dtype: float64

In [49]:
rec_df['Jace, the Mind Sculptor'].sort_values()[1:11]

name
Voyage's End             0.407243
Coral Fighters           0.408370
Select for Inspection    0.439047
Brainstorm               0.445725
Eye Spy                  0.460706
Anchor to the Aether     0.461245
Riverwise Augur          0.462028
Precognition             0.474661
Dream Cache              0.484562
Dissolve                 0.494091
Name: Jace, the Mind Sculptor, dtype: float64

In [50]:
rec_df['Delver of Secrets // Insectile Aberration'].sort_values()[1:11]

name
Think Tank                               0.276404
Puresight Merrow                         0.324103
Geist of the Archives                    0.332278
Aberrant Researcher // Perfected Form    0.346568
Etherwrought Page                        0.356932
Precognition Field                       0.366618
Galvanoth                                0.382904
Rummaging Wizard                         0.384269
Into the Wilds                           0.402021
Mudbutton Clanger                        0.406630
Name: Delver of Secrets // Insectile Aberration, dtype: float64

In [51]:
rec_df['Grizzly Bears'].sort_values()[1:11] # vanilla creature

name
Runeclaw Bear        0.036885
Bear Cub             0.081373
Balduvian Bears      0.081373
Forest Bear          0.081373
Alpine Grizzly       0.100436
Cylian Elf           0.112068
Norwood Ranger       0.124277
Elvish Warrior       0.125747
Charging Badger      0.155043
Swordwise Centaur    0.155350
Name: Grizzly Bears, dtype: float64

In [52]:
rec_df['Oko, Thief of Crowns'].sort_values()[1:11]

name
Bake into a Pie          0.377825
Fell the Pheasant        0.416711
Bartered Cow             0.421231
Savvy Hunter             0.431345
Fierce Witchstalker      0.445634
Wolf's Quarry            0.445707
Tempting Witch           0.458002
Fortifying Provisions    0.458092
Shrewd Negotiation       0.465474
Foreboding Fruit         0.495526
Name: Oko, Thief of Crowns, dtype: float64

In [53]:
rec_df['Gaze of Granite'].sort_values()[1:11]

name
Pernicious Deed      0.203794
Forced March         0.216513
Meltdown             0.231086
Displacement Wave    0.293787
Hammer Mage          0.382480
Dominate             0.401848
Granulate            0.436873
Disembowel           0.445209
Villainous Wealth    0.449117
Stir the Grave       0.460015
Name: Gaze of Granite, dtype: float64

In [54]:
rec_df['Tarmogoyf'].sort_values()[1:11]

name
Lhurgoyf                0.151893
Swarm of Rats           0.355881
Coiling Woodworm        0.363005
Wilderness Elemental    0.373945
Yavimaya Kavu           0.386207
Shambling Suit          0.402943
Dakmor Sorceress        0.407141
People of the Woods     0.407825
Treefolk Seedlings      0.413296
Sylvan Yeti             0.417179
Name: Tarmogoyf, dtype: float64

In [55]:
rec_df['Jace, Vryn\'s Prodigy // Jace, Telepath Unbound'].sort_values()[1:11]

name
Sins of the Past        0.460659
Kess, Dissident Mage    0.479928
Finale of Promise       0.497180
Bösium Strip            0.497745
Dreadhorde Arcanist     0.515003
Jaya Ballard            0.517537
Sphinx's Tutelage       0.520607
Diluvian Primordial     0.523249
Torrential Gearhulk     0.523376
Bag of Holding          0.534285
Name: Jace, Vryn's Prodigy // Jace, Telepath Unbound, dtype: float64

In [56]:
rec_df['Fatal Push'].sort_values()[1:11]

name
Fragmentize              0.506554
Renegade Rallier         0.519229
Granulate                0.543484
Thoughtbind              0.546189
Threads of Disloyalty    0.561036
Wretched Banquet         0.561761
Overload                 0.562909
Despark                  0.570954
Smother                  0.583768
Disembowel               0.595013
Name: Fatal Push, dtype: float64

In [57]:
rec_df['Veil of Summer'].sort_values()[1:11]

name
Autumn's Veil           0.309814
Display of Dominance    0.353770
Lazotep Plating         0.438167
Veilstone Amulet        0.462696
Spellbane Centaur       0.470685
Blinding Fog            0.486111
Join Shields            0.511953
Tortoise Formation      0.524454
Skyshroud Blessing      0.526529
Leonin Abunas           0.527038
Name: Veil of Summer, dtype: float64

In [58]:
rec_df['Urza, Lord High Artificer'].sort_values()[1:11]

name
Oracle's Vault         0.503835
Mind's Desire          0.508000
Temporal Aperture      0.532833
Aerial Caravan         0.539252
Karn, Scion of Urza    0.542545
Abbot of Keral Keep    0.546390
Djinn of Wishes        0.563756
Stolen Goods           0.574252
Outpost Siege          0.577444
Vital Splicer          0.593106
Name: Urza, Lord High Artificer, dtype: float64

In [59]:
converted_df.head()

,+0/+1,+0/+1 until,+0/+1 until end,+0/+1 until end of,+0/+1 until end of turn,+0/+2,+0/+3,+1,+1/+0,+1/+0 and,+1/+0 and gains,+1/+0 and has,+1/+0 for,+1/+0 for each,+1/+0 until,+1/+0 until end,+1/+0 until end of,...,legalities_pauper,legalities_future,legalities_brawl,legalities_modern,legalities_penny,legalities_vintage,rarity_common,rarity_uncommon,rarity_rare,rarity_mythic,power,power_back,toughness,toughness_back,loyalty,loyalty_back,cmc
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,-0.171423
Sensory Deprivation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,-1.300191
Road of Return,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,-0.735807
Storm Crow,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,-0.901216,0.0,-0.41462,0.0,0.0,0.0,-0.735807
Walking Sponge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,-0.901216,0.0,-0.99908,0.0,0.0,0.0,-0.735807


In [60]:
rec_df.head()

name,Static Orb,Sensory Deprivation,Road of Return,Storm Crow,Walking Sponge,Ravnica at War,Torrent of Fire,Pteramander,Nantuko Elder,Vedalken Heretic,Waterknot,Ruthless Knave,"Hua Tuo, Honored Physician",Veil of Summer,Disposal Mummy,Marang River Prowler,Aura Graft,...,Trinket Mage,Skyshroud Blessing,"Omnath, Locus of the Roil",Harvest Hand // Scrounged Scythe,Polis Crusher,Test of Endurance,Venom Sliver,Borderland Ranger,Curse of Thirst,Temporary Truce,Clearwater Goblet,Quarry Beetle,Devoted Hero,Without Weakness,Firesong and Sunspeaker,"Samut, the Tested",Sinew Sliver
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.000000,0.873256,0.922700,0.902261,0.902714,0.776722,0.902974,0.936812,0.834946,0.879136,0.845122,0.914283,0.909250,0.920169,0.900858,0.736824,0.903822,...,0.948871,0.903492,0.946999,0.755334,0.916317,0.878582,0.914390,0.947983,0.931651,0.823542,0.929009,0.929058,0.807054,0.934629,0.922868,0.903576,0.858726
Sensory Deprivation,0.873256,0.000000,0.921416,0.740269,0.753184,0.743237,0.899692,0.800355,0.732003,0.777973,0.509072,0.834738,0.901527,0.910635,0.761418,0.798380,0.755995,...,0.876959,0.862302,0.915397,0.785160,0.874106,0.903407,0.772500,0.888395,0.817251,0.886838,0.939183,0.868650,0.630570,0.839691,0.896963,0.878733,0.713580
Road of Return,0.922700,0.921416,0.000000,0.954260,0.881166,0.886467,0.880287,0.881781,0.867445,0.868711,0.944494,0.944848,0.752530,0.865972,0.842397,0.830297,0.882519,...,0.777963,0.905112,0.837042,0.885204,0.897840,0.819342,0.915827,0.774108,0.903097,0.902026,0.888863,0.684497,0.880370,0.902046,0.923654,0.828494,0.917864
Storm Crow,0.902261,0.740269,0.954260,0.000000,0.783188,0.856448,0.929037,0.828272,0.778113,0.830431,0.849624,0.885632,0.918470,0.867444,0.846572,0.764191,0.867024,...,0.893395,0.867449,0.939793,0.735358,0.918832,0.935606,0.846428,0.917411,0.925078,0.919631,0.940725,0.926406,0.727157,0.901698,0.931513,0.937399,0.753760
Walking Sponge,0.902714,0.753184,0.881166,0.783188,0.000000,0.884652,0.879105,0.805143,0.649754,0.817888,0.856545,0.827009,0.784286,0.763053,0.852932,0.807806,0.777807,...,0.865633,0.680594,0.873182,0.859729,0.916414,0.869660,0.815832,0.895287,0.833769,0.918551,0.896560,0.905671,0.691195,0.773115,0.916976,0.761958,0.763824


In [61]:
converted_df.to_csv('../Data/converted_df.csv')

In [62]:
rec_df.to_csv('../Data/recommender_df.csv')

In [87]:
df['cmc'].head()

name
Static Orb             3.0
Sensory Deprivation    1.0
Road of Return         2.0
Storm Crow             2.0
Walking Sponge         2.0
Name: cmc, dtype: float64

In [88]:
# need to drop supertype, 

drop_list = list(vt_list) + ['rarity_common', 'rarity_uncommon', 'rarity_rare', 'rarity_mythic',
                            'activated_ability', 'triggered_ability', 'cmc', 'power', 'power_back', 'toughness',
                            'toughness_back', 'loyalty', 'loyalty_back', 'super_type_Basic', 'super_type_World',
                            'super_type_Legendary', 'super_type_Snow']
filter_df = converted_df.drop(columns=drop_list)
filter_df['cmc'] = df['cmc']
filter_df.head()

,colors_B,color_identity_B,colors_G,color_identity_G,colors_R,color_identity_R,colors_U,color_identity_U,colors_W,color_identity_W,card_type_Creature,card_type_Instant,card_type_Enchantment,card_type_Sorcery,card_type_Artifact,card_type_Land,card_type_Planeswalker,...,sub_type_Horse,sub_type_Dovin,sub_type_Frog,legalities_standard,legalities_pioneer,legalities_commander,legalities_oldschool,legalities_duel,legalities_legacy,legalities_historic,legalities_pauper,legalities_future,legalities_brawl,legalities_modern,legalities_penny,legalities_vintage,cmc
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Static Orb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
Sensory Deprivation,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
Road of Return,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
Storm Crow,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,2.0
Walking Sponge,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


### SQL
I can't have an actual online live demo because my tables are too large to host for free. So, I'll being setting up a local version of what the app would be if I could get it to be live. For a local version I'm going to be using SQLite